# **Step 1: Install Required Packages**

In [2]:
!pip install git+https://github.com/Unsloth-ML/unsloth.git
!pip install transformers datasets accelerate peft bitsandbytes evaluate


  Cloning https://github.com/Unsloth-ML/unsloth.git to /tmp/pip-req-build-7zigblo9
  Running command git clone --filter=blob:none --quiet https://github.com/Unsloth-ML/unsloth.git /tmp/pip-req-build-7zigblo9
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/Unsloth-ML/unsloth.git /tmp/pip-req-build-7zigblo9 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/Unsloth-ML/unsloth.git /tmp/pip-req-build-7zigblo9 did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [3]:
pip install --upgrade transformers accelerate bitsandbytes


In [6]:
!pip install -U transformers

# **Step 2: Choose a Small Base Model**

In [15]:
from transformers import pipeline

model_name = "togethercomputer/Cerebras-GPT-2.7B"

pipe = pipeline(
    "text-generation",
    model=model_name,
    device_map="auto",  # GPU if available
    max_new_tokens=150
)

prompt = "Explain the common symptoms of diabetes in simple terms."
output = pipe(prompt)
print(output[0]['generated_text'])


OSError: togethercomputer/Cerebras-GPT-2.7B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

# **Step 3: Load a Domain-Specific Medical Dataset**

In [16]:
from datasets import load_dataset
import pandas as pd

# Example: Hugging Face clinical QA dataset
# dataset = load_dataset("medmcqa", split="train[:5000]")  # small subset

# Or use a custom CSV with columns: "prompt" and "completion"
df = pd.read_csv("/content/medical_qa.csv")
train_data = df.to_dict(orient="records")


OSError: togethercomputer/Cerebras-GPT-2.7B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

# **Step 4: Tokenize the Dataset**

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["prompt"], examples["completion"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )

tokenized_data = [tokenizer(d["prompt"], d["completion"], truncation=True, max_length=512) for d in train_data]


# **Step 5: Setup QLoRA Adapter with PEFT**

In [ ]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
lora_config = LoraConfig(
    r=16,                     # rank of low-rank matrices
    lora_alpha=32,            # scaling factor
    target_modules=["q_proj", "v_proj"],  # typically attention projections
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


# **Step 6: Training Setup**

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="./medical_llm_qlora",
    per_device_train_batch_size=1,  # small batch for free GPU
    gradient_accumulation_steps=8,  # simulate larger batch
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2
)

data_collator = DataCollatorForSeq2Seq(tokenizer, padding=True)


# **Step 7: Start Training**

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


# **Step 9: Test Fine-Tuned Model**

In [ ]:
from peft import PeftModel
import torch

# Load base + adapter
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, "./medical_llm_qlora_adapter")

# Test prompt
prompt = "What are the common symptoms of Type 2 Diabetes?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
